In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [ ]:
X = pd.read_csv('data/train/')
y =